# LSTM

### Tipos de LSTM:
- Univariate LSTM Models:

Vanilla LSTM

Stacked LSTM

Bidirectional LSTM

CNN LSTM

ConvLSTM


- Multivariate LSTM Models
- Multi-Step LSTM Models
- Multivariate Multi-Step LSTM Models

### Importando bibliotecas

In [55]:
import numpy as np

### Data preparations

In [78]:
# split sequencia de univariavel em varios samples
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # encontrar o fim desse padrão
        end_ix = i + n_steps
        # check se está além dessa sequência
        if end_ix > len(sequence)-1:
            break
        # inputs e outputs do padrão
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)
 
# ddefine sequencia de entrada
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
# escolhe o número de time steps
n_steps = 3
# chama a função
X, y = split_sequence(raw_seq, n_steps)
# mostra os dados
for i in range(len(X)):
    print(X[i], y[i])

[10 20 30] 40
[20 30 40] 50
[30 40 50] 60
[40 50 60] 70
[50 60 70] 80
[60 70 80] 90


In [79]:
# Definição do inputShape e ajustando o X
n_steps = 3
n_features = 1

# [samples, timesteps, features]
X = X.reshape(X.shape[0], n_steps, n_features)

In [58]:
X.shape

(6, 3, 1)

## Valina LSTM

- Tem apenas uma layers oculta e uma layer de saída

#### Definição do Modelo Valina LSTM univariavel

In [59]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional, TimeDistributed, Conv1D, MaxPooling1D, Flatten

In [60]:
# Definição do modelo
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')

In [61]:
model.fit(X, y, epochs = 100, verbose=0)

### Fazendo previsão

In [62]:
x_input = np.array([70,80,90])
x_input = x_input.reshape((1, n_steps,n_features))

In [63]:
yhat = model.predict(x_input)

In [64]:
print(yhat)

[[102.36032]]


## Stacked LSTM

- Multiplas layers ocultas e uma layer de saída

- Para colocar varias camadas sequenciais, temos que colocar o parâmetro na camada anterior:
    return_sequences=True
#### Definição do Modelo Stacked LSTM univariavel

In [65]:
model = Sequential()

model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
model.add(LSTM(50, activation='relu'))

model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')

In [66]:
model.fit(X, y, epochs=100, verbose=0)

### Fazendo previsão


In [67]:
x_input = np.array([70,80,90])
x_input = x_input.reshape(1, n_steps, n_features)

In [68]:
yhat = model.predict(x_input)

In [69]:
print(yhat)

[[102.48096]]


## Bidirecional LSTM

- aprenda a seqüência de entrada tanto para frente quanto para trás e concatene ambas as interpretações.

- a primeira camada oculta em uma camada de wrapper chamada Bidirecional.

#### Definição do Modelo Bidirecional LSTM 

In [70]:
model = Sequential()

model.add(Bidirectional(LSTM(50, activation='relu'), input_shape=(n_steps, n_features)))

model.add(Dense(1))

model.compile(optimizer='adam', loss='mse')

In [71]:
model.fit(X, y, epochs=100, verbose=0)

### Fazendo previsão

In [72]:
x_input = np.array([70,80,90])
x_input = x_input.reshape(1, n_steps, n_features)

In [73]:
yhat = model.predict(x_input)

In [74]:
print(yhat)

[[102.14232]]


## CNN LSTM

- CNN com LSTM
#### Definição do Modelo CNN LSTM

In [75]:
# choose a number of time steps
n_steps = 4

# split into samples
X, y = split_sequence(raw_seq, n_steps)

# reshape from [samples, timesteps] into [samples, subsequences, timesteps, features]
n_features = 1
n_seq = 2
n_steps = 2
X = X.reshape((X.shape[0], n_seq, n_steps, n_features))

# define model
model = Sequential()
model.add(TimeDistributed(Conv1D(filters=64, kernel_size=1, activation='relu'), input_shape=(None, n_steps, n_features)))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# fit model
model.fit(X, y, epochs=500, verbose=0)

# demonstrate prediction
x_input = np.array([60, 70, 80, 90])
x_input = x_input.reshape((1, n_seq, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[103.00484]]


### Exemplo simples

In [85]:
# Example of LSTM to learn a sequence
from pandas import DataFrame
from pandas import concat
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
# create sequence
length = 10
sequence = [i/float(length) for i in range(length)]
print(sequence)
# create X/y pairs
df = DataFrame(sequence)
df = concat([df.shift(1), df], axis=1)
df.dropna(inplace=True)
# convert to LSTM friendly format
values = df.values
X, y = values[:, 0], values[:, 1]
X = X.reshape(len(X), 1, 1)
print("y:", y)
# 1. define network
model = Sequential()
model.add(LSTM(10, input_shape=(1,1)))
model.add(Dense(1))
# 2. compile network
model.compile(optimizer='adam', loss='mean_squared_error')
# 3. fit network
history = model.fit(X, y, epochs=1000, batch_size=len(X), verbose=0)
# 4. evaluate network
loss = model.evaluate(X, y, verbose=0)
print(loss)
# 5. make predictions
predictions = model.predict(X, verbose=0)
print(predictions[:, 0])

[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
y: [0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9]
7.350883970502764e-05
[0.12205631 0.20999527 0.3020866  0.39757013 0.4956392  0.5954754
 0.69628197 0.79731363 0.8979009 ]


In [89]:
y - predictions[:,0]

array([-0.02205631, -0.00999527, -0.00208659,  0.00242987,  0.0043608 ,
        0.00452462,  0.00371803,  0.00268637,  0.00209912])